In [1]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Thu Apr  4 14:53:08 2019

@author: marcos
"""

'''Example of VAE on MNIST dataset using CNN

The VAE has a modular design. The encoder, decoder and VAE
are 3 models that share weights. After training the VAE model,
the encoder can be used to  generate latent vectors.
The decoder can be used to generate MNIST digits by sampling the
latent vector from a Gaussian distribution with mean=0 and std=1.

# Reference

[1] Kingma, Diederik P., and Max Welling.
"Auto-encoding variational bayes."
https://arxiv.org/abs/1312.6114
'''


from keras.layers import Dense, Input
from keras.layers import Conv2D, Flatten, Lambda
from keras.layers import Reshape, Conv2DTranspose, Cropping2D
from keras.models import Model
from keras.datasets import mnist
from keras.losses import mse, binary_crossentropy
from keras.utils import plot_model
from keras import backend as K

import numpy as np
import matplotlib.pyplot as plt
import argparse
import os
import pandas as pd

# reparameterization trick
# instead of sampling from Q(z|X), sample eps = N(0,I)
# then z = z_mean + sqrt(var)*eps
def sampling(args):
    """Reparameterization trick by sampling fr an isotropic unit Gaussian.

    # Arguments
        args (tensor): mean and log of variance of Q(z|X)

    # Returns
        z (tensor): sampled latent vector
    """

    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon


def plot_results(models,
                 data,
                 batch_size=128,
                 model_name="vae_bussim"):
    """Plots labels and MNIST digits as function of 2-dim latent vector

    # Arguments
        models (tuple): encoder and decoder models
        data (tuple): test data and label
        batch_size (int): prediction batch size
        model_name (string): which model is using this function
    """

    encoder, decoder = models
    x_test, y_test = data
    os.makedirs(model_name, exist_ok=True)

    filename = os.path.join(model_name, "vae_mean.png")
    # display a 2D plot of the digit classes in the latent space
    z_mean, _, _ = encoder.predict(x_test,
                                   batch_size=batch_size)
    plt.figure(figsize=(12, 10))
    plt.scatter(z_mean[:, 0], z_mean[:, 1], c=y_test)
    plt.colorbar()
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.savefig(filename)
    plt.show()

    filename = os.path.join(model_name, "digits_over_latent.png")
    # display a 30x30 2D manifold of digits
    n = 30
    digit_size = 28
    figure = np.zeros((digit_size * n, digit_size * n))
    # linearly spaced coordinates corresponding to the 2D plot
    # of digit classes in the latent space
    grid_x = np.linspace(-4, 4, n)
    grid_y = np.linspace(-4, 4, n)[::-1]

    for i, yi in enumerate(grid_y):
        for j, xi in enumerate(grid_x):
            z_sample = np.array([[xi, yi]])
            x_decoded = decoder.predict(z_sample)
            digit = x_decoded[0].reshape(digit_size, digit_size)
            figure[i * digit_size: (i + 1) * digit_size,
                   j * digit_size: (j + 1) * digit_size] = digit

    plt.figure(figsize=(10, 10))
    start_range = digit_size // 2
    end_range = n * digit_size + start_range + 1
    pixel_range = np.arange(start_range, end_range, digit_size)
    sample_range_x = np.round(grid_x, 1)
    sample_range_y = np.round(grid_y, 1)
    plt.xticks(pixel_range, sample_range_x)
    plt.yticks(pixel_range, sample_range_y)
    plt.xlabel("z[0]")
    plt.ylabel("z[1]")
    plt.imshow(figure, cmap='Greys_r')
    plt.savefig(filename)
    plt.show()

    
selected_line =['8700-10-1',
                '7545-10-1',
                '7545-10-0',   #Less two links
                '6450-10-1',
                '6450-10-0',
                '3301-10-1',   #Less two links
                '2290-10-1',
                '2290-10-0',
                '477P-10-0',        
                '3301-10-0',   #Less two links
                '574J-10-1',   #Less two links
                '574J-10-0',   #Less two links
                '477P-10-1',   #Less two links
                '351F-10-1',
                '351F-10-0'] 

selected_line = ['6450-10-0']

### Size of the steps to group
frequencies = ['20min', '30min', '1H', '3H', '1d', '7d', '1m']
frequencies = ['30min']


for line in selected_line:  
    filename = str(line) + '_temp.csv.gz'
    df = pd.read_csv(filename, compression='gzip', sep=',')
    df['exact_time'] = pd.to_datetime(df['exact_time'], format = '%Y-%m-%d %H:%M')
    df.index = df['exact_time']
    
    df.loc[(df['time_link'] > 5),'time_link'] = np.ceil(df['time_link'].mean())
    
    start_date = pd.to_datetime('2017-1-1', format = '%Y-%m-%d')
    end_date = pd.to_datetime('2017-8-30', format = '%Y-%m-%d')
    df = df.loc[(df['holiday'] != 1) & ((df['weekday'] > 0) & (df['weekday'] < 5))]
 
    
    frequency = '30min'            
    rolling_win = 1
    df = df.drop(df[df['link'] == max(df['link'])].index)
    if (line == '7545-10-1') | (line == '477P-10-1') | (line == '3301-10-0') | \
            (line == '3301-10-0') | (line == '574J-10-1') | (line == '574J-10-1'):
            df = df.drop(df[df['link'] == max(df['link'])].index)
            
    X_Temp = df.groupby([pd.Grouper(freq=str(frequency)), 'link'], as_index=True ).median()['time_link'].unstack()    
    X_Temp = X_Temp.transform(lambda x: x.fillna(method='ffill')).dropna()

    X_Temp = X_Temp.iloc[X_Temp.index.indexer_between_time('08:00', '22:00')]

    from sklearn import mixture
    gmm = mixture.GaussianMixture(n_components=5, covariance_type='full')


    
   

Using TensorFlow backend.


In [187]:
gmm.fit(X_Temp[])
gmm.means_ = np.reshape(np.array(X_Temp[:1]), 29)
gmm.sample(100)

GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=100,
                means_init=None, n_components=5, n_init=1, precisions_init=None,
                random_state=None, reg_covar=1e-06, tol=0.001, verbose=0,
                verbose_interval=10, warm_start=False, weights_init=None)

In [338]:
gmm.means_[np.argmax(gmm.predict_proba(X_Temp[:1]))] = np.reshape(np.array(X_Temp[:1]), 29)

np.reshape(gmm.sample()[0], 29)[0]


3.653645788926031

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import heapq 
import pickle
%matplotlib inline
import matplotlib.pyplot as plt
import dateutil
import datetime 
from time import time
import re
from scipy import interpolate
import timeit
from dateutil.rrule import DAILY, rrule, MO, TU, WE, TH, FR
import seaborn as sns

from sklearn.metrics import mean_absolute_error
import math

import sys
sys.path.append("../")


from historical.readData.old.estimateData3 import read

from historical.readData.old.estimateData3 import read
from historical.readData.old.estimateData3 import search_travels
from historical.readData.old.estimateData3 import estimate
from historical.readData.old.estimateData3 import stops_distance

from historical.readData.old.travels3 import haversine2

def calcula_dist_shape(selec_linhas):
    distance_all_shapes = {}
    # print 'Calculando distancias dos shapes'
    '''calcula todas as distancias dos shapes'''
    
    pth_files_GTFS = "../historical/readData/dados/gtfs/"        
        
    trips = pd.read_csv(pth_files_GTFS + 'trips.txt', sep=',')
    shapes = pd.read_csv(pth_files_GTFS + 'shapes.txt', sep=',')
    
    for l, trip_id in enumerate(selec_linhas):

        trip = trips[trips.trip_id == trip_id]
        trip_shape = shapes[shapes['shape_id'].isin(trip['shape_id'])]
        shapelat = trip_shape.shape_pt_lat.tolist()
        shapelon = trip_shape.shape_pt_lon.tolist()

        # distancias dos shapes
        lon1 = shapelon[0]
        lat1 = shapelat[0]
        totalcal = [0.]
        dist = [0.]
        for lat2, lon2 in zip(shapelat[1:], shapelon[1:]):
            d = haversine2(lat1, lon1, lat2, lon2)
            d = d * 1000
            dist.append(d)
            totalcal.append(totalcal[-1] + d)
            lat1 = lat2
            lon1 = lon2
        distance_all_shapes[trip_id] = [shapelat, shapelon, totalcal]
    return distance_all_shapes

def stops_distance(linha):

    distances = calcula_dist_shape([linha])
    totalcal = distances[linha][2]
    
    pth_files_GTFS = "../historical/readData/dados/gtfs/"
    

    trips = pd.read_csv(pth_files_GTFS + 'trips.txt', sep=',')
    shapes = pd.read_csv(pth_files_GTFS + 'shapes.txt', sep=',')
    stops = pd.read_csv(pth_files_GTFS + 'stops.txt', sep=',')
    stopid = pd.read_csv(pth_files_GTFS + 'stop_times.txt', sep=',')   

    ida = trips[trips.trip_id == linha]
    shapeida = shapes[shapes['shape_id'].isin(ida['shape_id'])]
    idalat = shapeida.shape_pt_lat.tolist()
    idalon = shapeida.shape_pt_lon.tolist()
    total = shapeida.shape_dist_traveled.tolist()

    temp1 = stopid[stopid.trip_id == linha]
    stopsida = stops[stops['stop_id'].isin(temp1['stop_id'])]
    stopsida = stopsida.set_index('stop_id')
    stopsida = stopsida.reindex(index=temp1['stop_id'])

    stopslat = stopsida.stop_lat.tolist()
    stopslon = stopsida.stop_lon.tolist()

    dpontos = [None] * len(stopslat)
    index = 0
    lat = idalat
    lon = idalon
    total = 0
    p = ['depois'] * len(stopslat)
    for latb, lonb, i in zip(stopslat, stopslon, range(len(stopslat))):
        lat = lat[index:]
        lon = lon[index:]
        nn = haversine2(latb, lonb, np.array(lat), np.array(lon)) * 1000
        index = nn.argmin()
        total = total + index
        if index == 0:
            dpontos[i] = nn[index]
#            descontardist = nn[index]
        else:
            if totalcal[total] >= totalcal[total - 1] + nn[index - 1]:
                p[i] = 'antes'
            dpontos[i] = totalcal[total - 1] + nn[index - 1]

    mid_points = []
#    mid_points.append(200)
    p1 = dpontos[0]
    for p2 in dpontos[1:]:
        mid_points.append(p1 + (p2-p1)/2)
        p1 = p2
    dpontos = [i/1000. for i in dpontos]
    mid_points = [i/1000. for i in mid_points]   

    return dpontos, mid_points

def read(filedata,filerep):
    df0 = pd.read_pickle(filedata, compression=None)
    with open(filerep, 'rb') as handle:
        reps = pickle.load(handle, encoding='latin1')
    return df0, reps

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + datetime.timedelta(n)
        
def daterangeWD(start_date, end_date):
  return rrule(DAILY, dtstart=start_date, until=end_date, byweekday=(MO,TU,WE,TH,FR))

def hr_func(ts):
    return ts.hour

def minute_func(ts):
    return ts.minute

def second_func(ts):
    return ts.second

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

import warnings
warnings.filterwarnings('ignore')


# if __name__ == '__main__':

selected_line = ['8700-10-1']

selected_line = ['8700-10-1',                
                '7545-10-1',
                '7545-10-0',
                '6450-10-1',
                '6450-10-0',
                '3301-10-1',
#                 '3301-10-0',
                '2290-10-1',
                '2290-10-0',
#                 '574J-10-1',
#                 '574J-10-0',
#                 '477P-10-1',
                '477P-10-0',
#                 '351F-10-1',
#                 '351F-10-0'
                ]

pth_files_GTFS = "../historical/readData/dados/gtfs/"

trips = pd.read_csv(pth_files_GTFS + 'trips.txt', sep=',')
shapes = pd.read_csv(pth_files_GTFS + 'shapes.txt', sep=',')
stops = pd.read_csv(pth_files_GTFS + 'stops.txt', sep=',')
stopid = pd.read_csv(pth_files_GTFS + 'stop_times.txt', sep=',')   

periods = ['morning', 'm_peak', 'i_peak', 'a_peak', 'night']

# for line in selected_line:
line = selected_line[0]
p, mp = stops_distance(line)
pth_files_lines = "../historical/readData/"


filename = str(line) + '_temp.csv.gz'
df_training = pd.read_csv(filename, compression='gzip', sep=',')
df_training['exact_time'] = pd.to_datetime(df_training['exact_time'], format = '%Y-%m-%d %H:%M')
df_training.index = df_training['exact_time']

#     df.loc[(df['time_link'] > 5),'time_link'] = np.ceil(df['time_link'].mean())

start_date = pd.to_datetime('2017-8-1', format = '%Y-%m-%d')
end_date = pd.to_datetime('2017-8-30', format = '%Y-%m-%d')

df_training = df_training.loc[((df_training['exact_time'] >= start_date) & (df_training['exact_time'] <= end_date) & 
                        (df_training['holiday'] != 1)) & ((df_training['weekday'] != 6) & 
                        (df_training['weekday'] != 5) & (df_training['weekday'] != 4) & 
                        (df_training['weekday'] != 0))]

#     df_training = df_training.loc[(df['holiday'] != 1) & ((df['weekday'] > 0) & (df['weekday'] < 5))]


frequency = '30min'            
rolling_win = 1
df_training = df_training.drop(df_training[df_training['link'] >= max(df_training['link']) - 1].index)

#     if (line == '7545-10-1') | (line == '477P-10-1') | (line == '3301-10-0') | \
#             (line == '3301-10-0') | (line == '574J-10-1') | (line == '574J-10-1'):
#             df_training = df_training.drop(df_training[df_training['link'] == max(df_training['link'])].index)

X_Temp = df_training.groupby([pd.Grouper(freq=str(frequency)), 'link'], as_index=True ).median()['time_link'].unstack()    
X_Temp = X_Temp.transform(lambda x: x.fillna(method='ffill')).dropna()

X_Temp = X_Temp.iloc[X_Temp.index.indexer_between_time('08:00', '22:00')]

from sklearn import mixture
gmm = mixture.GaussianMixture(n_components=5, covariance_type='full')
gmm.fit(X_Temp)

### Operação demorada
df, reps = read(pth_files_lines + "trips_" + line + ".dsk", pth_files_lines + "interps_" + line + ".rep")

df['day'] = pd.to_datetime(df['day'], format = '%Y-%m-%d')
df['start'] = pd.to_datetime(df['start'], format = '%H:%M:%S')

end_date = datetime.date(2017, 9, 1)        

df = df.loc[((df['day'] >= end_date) & (df['holiday'] != 1)) & ((df['weekday'] != 6) & (df['weekday'] != 5) & (df['weekday'] != 4) & (df['weekday'] != 0))]

df['exact_time'] = np.array(df['day'], dtype='datetime64[ns]') + \
                    pd.to_timedelta(pd.to_timedelta(pd.DatetimeIndex(df['start']).hour*60 + \
                    pd.DatetimeIndex(df['start']).minute + \
                    pd.DatetimeIndex(df['start']).second/60 + \
                    df['time'], unit='m'))

df.index = df['exact_time']
df.iloc[df.index.indexer_between_time('08:00', '20:00')]

df['day_hour'] = df['exact_time'].apply(hr_func)

df['link'] = 0
for i in range(0, len(mp)):
    if (i == 0):
          df.loc[df.loc[(df['distance'] > 0) & (df['distance'] < mp[i+1]),]['link'].index,'link'] = i    
    if (i == len(mp) -1):
        df.loc[df.loc[(df['distance'] > mp[i]) & (df['distance'] < max(p)),]['link'].index,'link'] = i    
    if (i != 0) & (i != len(mp)-1):
        df.loc[df.loc[(df['distance'] >= mp[i]) & (df['distance'] < mp[i+1]),]['link'].index,'link'] = i    

df = df.drop_duplicates(subset=['day', 'numtravel', 'link', 'day_hour'])   

link_df = []
travels = sorted(list(set(df.numtravel.unique())))
for tr in travels:
    tck = reps[tr][0]
    tck_mods = [(tck[0],tck[1]-m,tck[2]) for m in mp]       
    tempo = [a[0] if a.size>0 else np.nan for a in [interpolate.sproot(tck_mod) for tck_mod in tck_mods]]
    row = [tuple([tempo[i+1]-pos for i, pos in enumerate(tempo[:-1])])]    
    link_df.append(row)

travel_times = pd.DataFrame(np.reshape(link_df, (np.shape(link_df)[0], np.shape(link_df)[2])))
travel_times.index = travels

MAPE_array = []

selected_day = df.day.dt.strftime('%Y-%m-%d').unique()


df = df.drop(df[df['link'] >= max(df_training['link']) ].index)

In [4]:

for select_day in df.day.dt.strftime('%Y-%m-%d').unique():
    for link in range(1, len(mp)-2):
        df_temp = df.loc[(df['day'] == select_day)]
        # travels_day = np.unique(np.array(df_temp['numtravel']))

        for hour_day in range(640, 1320, 30):   
            int_part, float_part = divmod((hour_day/60), 1)

            if float_part != 0:
                hour_begin = str(int(int_part)) + ':' + '30'
                hour_end = str(int(int_part + 1 )) + ':' + '00'
                hour_test = str(int(int_part + 1 )) + ':' + '30'
            else:
                hour_begin = str(int(int_part)) + ':' + '00'
                hour_end = str(int(int_part)) + ':' + '30'
                hour_test = str(int(int_part + 1 )) + ':' + '00'

            df_temp_time = df_temp.iloc[df_temp.index.indexer_between_time( hour_begin, hour_end)]
            df_temp_test = df_temp.iloc[df_temp.index.indexer_between_time( hour_end, hour_test)]

            median_training = np.nanmedian(travel_times.loc[np.unique(np.array(df_temp_time['numtravel']))], axis=0)

            gmm.means_[np.argmax(gmm.predict_proba(median_training.reshape(1,max(df_training['link'])+1)))] = np.reshape(np.array(median_training), max(df_training['link']) )

            median_test = np.reshape(gmm.sample()[0], len(mp))[link]
            time_tests = travel_times.loc[np.unique(np.array(df_temp_test['numtravel']))][link]
            if (len(time_tests) > 0) & (math.isnan(median_test) == False) & (np.isnan(time_tests).any() == False):
                MAPE_array.append(mean_absolute_percentage_error([median_test]*len(time_tests), time_tests))

MAPE_array = np.array(MAPE_array)
print(line + ' MAPE(all) = ' + str(format(np.mean(MAPE_array), '.2f')) + '%')


ValueError: cannot reshape array of size 36 into shape (34,)

In [445]:
max(df_training['link']) 

link,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
exact_time,,,,,,,,,,,,,,,,,,,,,
2017-08-01 08:00:00,3.363464,2.631324,2.111979,1.781047,1.603638,2.028762,2.142451,1.844050,1.854569,2.290961,...,1.021972,1.139130,1.716724,1.412313,0.867300,0.743069,0.755811,0.967137,1.037147,1.376094
2017-08-01 08:30:00,3.574680,2.802002,2.298239,1.896906,1.710751,2.126025,2.294967,1.893627,1.990705,2.493407,...,1.006628,1.138705,1.709637,1.429535,0.880011,0.778406,0.809612,1.016009,1.100079,1.370098
2017-08-01 09:00:00,3.037907,2.480573,2.102831,1.750642,1.593753,2.023884,2.228439,1.878467,1.970295,2.432158,...,1.037008,1.151318,1.701050,1.426990,0.872287,0.758456,0.803075,1.026328,1.128604,1.482292
2017-08-01 09:30:00,3.158719,2.521722,2.046399,1.792657,1.629807,2.056034,2.262592,1.937396,2.046101,2.528496,...,1.170061,1.312272,1.974994,1.614166,1.062856,0.908274,0.930016,1.111185,1.203508,1.598315
2017-08-01 10:00:00,2.645598,2.380761,2.035916,1.719094,1.556466,1.970436,2.177619,1.824136,1.908371,2.396489,...,1.203572,1.340079,2.015086,1.591675,0.994699,1.028408,0.942465,1.319440,1.429437,1.895914
2017-08-01 10:30:00,2.869187,2.220912,1.861000,1.582893,1.434520,1.848706,2.082728,1.758922,1.860241,2.327461,...,1.132950,1.277118,1.982847,1.630801,1.027735,0.903292,0.924998,1.200784,1.313271,1.760738
2017-08-01 11:00:00,3.798901,2.761224,2.210763,1.733760,1.562269,1.955604,2.139773,1.773189,1.880752,2.319331,...,1.086860,1.242805,1.871242,1.534741,0.978655,0.854335,0.876116,1.123073,1.226232,1.645496
2017-08-01 11:30:00,4.526947,3.031108,2.264973,1.861820,1.660443,2.066177,2.247540,1.916478,2.003892,2.474251,...,1.124715,1.224482,1.937363,1.565127,0.993382,0.862392,0.881349,1.181806,1.239324,1.638893
2017-08-01 12:00:00,4.007421,2.981980,2.265233,1.846312,1.640437,2.049761,2.219395,1.853060,1.920686,2.357690,...,1.146204,1.284425,1.933518,1.580789,0.995202,0.848568,0.877057,1.152005,1.249381,1.699455
